# Lego Analysis

Description of task - TODO
Data from https://rebrickable.com/downloads/
## 💪 Challenge
Create a report to summarize your findings. Include:

1. What is the average number of Lego sets released per year?
2. What is the average number of Lego parts per year?
3. Create a visualization for item 2.
4. What are the 5 most popular colors used in Lego parts?
5. What proportion of Lego parts are transparent?
6. What are the 5 rarest lego bricks?
7. Summarize your findings.


In [1]:
# import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Prepare colors_table

In [2]:
colors_table = pd.read_csv('data\colors.csv.gz')
print(colors_table.head())
print('shape:', colors_table.shape)
print('unique names:', colors_table['name'].nunique())

   id            name     rgb is_trans
0  -1       [Unknown]  0033B2        f
1   0           Black  05131D        f
2   1            Blue  0055BF        f
3   2           Green  237841        f
4   3  Dark Turquoise  008F9B        f
shape: (217, 4)
unique names: 217


In [3]:
colors_table['is_trans_2'] = colors_table['is_trans'] == 't'
colors_table.drop('is_trans', axis='columns', inplace=True)
colors_table.rename(columns={'is_trans_2': 'is_trans'}, inplace=True)
colors_table.head()

,id,name,rgb,is_trans
0,-1,[Unknown],0033B2,False
1,0,Black,05131D,False
2,1,Blue,0055BF,False
3,2,Green,237841,False
4,3,Dark Turquoise,008F9B,False


In [4]:
print('unique names:', colors_table['is_trans'].unique())

unique names: [False  True]


In [5]:
colors_table.head()

,id,name,rgb,is_trans
0,-1,[Unknown],0033B2,False
1,0,Black,05131D,False
2,1,Blue,0055BF,False
3,2,Green,237841,False
4,3,Dark Turquoise,008F9B,False


Prepare elements

In [6]:
elements_table = pd.read_csv('data\elements.csv.gz')
print(elements_table.head())
print('shape:', elements_table.shape)

   element_id  part_num  color_id
0     6300211  67906c01        14
1     4566309      2564         0
2     4275423     53657      1004
3     6194308     92926        71
4     6229123     26561         4
shape: (77832, 3)


In [7]:
elements_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77832 entries, 0 to 77831
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   element_id  77832 non-null  int64 
 1   part_num    77832 non-null  object
 2   color_id    77832 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ MB


## QUESTION 1:
1. What is the average number of Lego sets released per year?


## QUESTION 2:
2. What is the average number of Lego parts per year?


## QUESTION 3:
3. Create a visualization for item 2.
